<style>
    h1 {
    font-family: 'Segoe UI Semibold';
    font-size: 35px;
    text-align: center;
  }
</style>

# Taller 9: Bases de Datos en la Nube (RDS)


<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
            object-position: top center;
        }
  .container {
            width: 100%;
            height: 350px;
        }

</style>
<div class="container">
    <img src= "./img/sql-cloud.png" class="image"/>
</div>

<style>
  p {
    font-family: 'Segoe UI Light';
    font-size: 20px;
    text-align: center;
  }
</style>

<div style="text-align: center;">
  Analítica Computacional para la Toma de Decisiones IIND-4130<br>
  Universidad de los Andes &copy; 2024 <br>
  21 de octubre de 2024 <br><br>
  <strong>GRUPO:</strong> <br>
  Camilo Duque - 202024289<br>
  Sofia Buitrago – 202020742<br>
</div>


---
***Evidencia Código de Python (camilo)**


### **Importar Librerias**

In [42]:
import psycopg2
import pandas as pd

### **Conectarse a Base de Datos (bd_postgres)**

In [43]:
# conexion a base de datos
engine = psycopg2.connect(
dbname="bd_postgres",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)

### **Conectarse a Base de Datos (World)**

In [44]:
# conexion a base de datos
engine = psycopg2.connect(
dbname="world",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)
# crear cursor
cursor = engine.cursor()

#### Consulta - Nombre de Tablas de la BD

In [45]:
# consulta
query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
    """
# ejecutar consulta
cursor.execute(query)

# extraer todas la filas de consulta
res = cursor.fetchall()

# imprimir tablas
print("Tablas en la base de datos:")
query_tbl = pd.DataFrame(res, columns=[cursor.description[0][0]])
query_tbl


Tablas en la base de datos:


,table_name
0,city
1,country
2,country_language


#### Consulta 1 - Estructura de tablas BD

In [46]:
try:
    # consulta
    query1 = """
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position; 
        """
    # ejecutar consulta
    cursor.execute(query1)

    # extraer todas la filas de consulta
    res1 = cursor.fetchall()
    
    # guardar en dataframe
    query1_estructura_bd = pd.DataFrame(res1, columns=[desc[0] for desc in cursor.description])

# Excepcion para revertir los cambios en caso de falla en el query
except Exception as e:
    print(f"Error: {e}")
    cursor.connection.rollback()

query1_estructura_bd

,table_name,column_name,data_type
0,city,id,integer
1,city,name,text
2,city,country_code,character
3,city,district,text
4,city,population,integer
5,country,code,character
6,country,name,text
7,country,continent,text
8,country,region,text
9,country,surface_area,real


#### Consulta 2 - Top 5 paises con más alto GNP per capita

In [47]:
try:
  # consulta
  query2 = """
      SELECT *, gnp/population AS gnp_per_capita
      FROM country
      WHERE population != 0
      ORDER BY gnp_per_capita DESC
      LIMIT 5; 
      """
  # ejecutar consulta
  cursor.execute(query2)
  # extraer todas la filas de consulta
  res2 = cursor.fetchall()
  # guardar en dataframe
  query2_gnp_per_capita = pd.DataFrame(res2, columns=[desc[0] for desc in cursor.description])

# Excepcion para revertir los cambios en caso de falla en el query
except Exception as e:
    print(f"Error: {e}")
    cursor.connection.rollback()
query2_gnp_per_capita

,code,name,continent,region,surface_area,indep_year,population,life_expectancy,gnp,gnp_old,local_name,government_form,head_of_state,capital,code2,gnp_per_capita
0,LUX,Luxembourg,Europe,Western Europe,2586.0,1867.0,435700,77.1,16321.00,15519.00,Luxembourg/Lëtzebuerg,Constitutional Monarchy,Henri,2452,LU,0.03745926095937571724
1,CHE,Switzerland,Europe,Western Europe,41284.0,1499.0,7160400,79.6,264478.00,256092.00,Schweiz/Suisse/Svizzera/Svizra,Federation,Adolf Ogi,3248,CH,0.03693620468130272052
2,BMU,Bermuda,North America,North America,53.0,NaN,65000,76.9,2328.00,2190.00,Bermuda,Dependent Territory of the UK,Elisabeth II,191,BM,0.03581538461538461538
3,BRN,Brunei,Asia,Southeast Asia,5765.0,1984.0,328000,73.6,11705.00,12460.00,Brunei Darussalam,Monarchy (Sultanate),Haji Hassan al-Bolkiah,538,BN,0.03568597560975609756
4,LIE,Liechtenstein,Europe,Western Europe,160.0,1806.0,32300,78.8,1119.00,1084.00,Liechtenstein,Constitutional Monarchy,Hans-Adam II,2446,LI,0.03464396284829721362


#### Consulta 3 - Top 10 ciudades de Sur America con más alto porcentaje de población con respecto al país

In [48]:
try:
  # consulta
  query3 = """
      SELECT
        city.name AS nombre_ciudad,
        country.name AS nombre_pais,
        city.population AS poblacion_ciudad,
        country.population AS poblacion_pais,
        (CAST(city.population AS FLOAT)/CAST(country.population AS FLOAT))*100 AS porc_pob
      FROM city
      LEFT JOIN country ON city.country_code = country.code
      WHERE continent = 'South America'
      ORDER BY porc_pOb DESC
      LIMIT 10; 
      """
  # ejecutar consulta
  cursor.execute(query3)
  # extraer todas las filas de consulta
  res3 = cursor.fetchall()
  # guardar dataframe
  query3_porc_pob = pd.DataFrame(res3, columns=[desc[0] for desc in cursor.description])
# Excepcion para revertir los cambios en caso de falla en el query
except Exception as e:
    print(f"Error: {e}")
    cursor.connection.rollback()
query3_porc_pob

,nombre_ciudad,nombre_pais,poblacion_ciudad,poblacion_pais,porc_pob
0,Stanley,Falkland Islands,1636,2000,81.800000
1,Montevideo,Uruguay,1236000,3337000,37.039257
2,Santiago de Chile,Chile,4703954,15211000,30.924686
3,Georgetown,Guyana,254000,861000,29.500581
4,Cayenne,French Guiana,50699,181000,28.010497
5,Paramaribo,Suriname,112000,417000,26.858513
6,Lima,Peru,6464693,25662000,25.191696
7,Guayaquil,Ecuador,2070040,12646000,16.369129
8,Santafé de Bogotá,Colombia,6260862,42321000,14.793748
9,Quito,Ecuador,1573458,12646000,12.442337


### **Conectarse a Base de Datos Sofia (World)**

In [49]:
engine = psycopg2.connect(
dbname="world",
user="postgres",
password="uNicentro17",
host="sofia.cmubcp0881c7.us-east-1.rds.amazonaws.com",
port='5432'
)
cursor = engine.cursor()

#### Consulta 1 - Idiomas más hablados en el Mundo

In [50]:
try:
    # consulta
    query1 = """
        SELECT language, SUM(percentage/100*population) AS poblacion_idioma
        FROM country
        LEFT JOIN country_language ON country.code = country_language.country_code
        WHERE percentage/100*population IS NOT NULL
        GROUP BY language
        ORDER BY poblacion_idioma DESC
        LIMIT 5; 
        """
    # ejecutar consulta
    cursor.execute(query1)
    # extraer todas la filas de consulta
    res1 = cursor.fetchall()
    # guardar en dataframe
    query1_idiomas = pd.DataFrame(res1, columns=[desc[0] for desc in cursor.description])

# Excepcion para revertir los cambios en caso de falla en el query
except Exception as e:
    print(f"Error: {e}")
    cursor.connection.rollback()

query1_idiomas

,language,poblacion_idioma
0,Chinese,1.191844e+09
1,Hindi,4.056331e+08
2,Spanish,3.550295e+08
3,English,3.470779e+08
4,Arabic,2.338392e+08


#### Consulta 2 - Continentes con Mayor Life Expectancy

In [51]:
try:
    # consulta
    query2 = """
    SELECT continent, AVG(life_expectancy) AS avg_life_expectancy
    FROM country
    WHERE life_expectancy IS NOT NULL
    GROUP BY continent
    ORDER BY avg_life_expectancy DESC;
    """
    # ejecutar consulta
    cursor.execute(query2)
    # extraer todas la filas de consulta
    res2 = cursor.fetchall()
    # guardar en dataframe
    query2_avg_life_continente = pd.DataFrame(res2, columns=[desc[0] for desc in cursor.description])

# Excepcion para revertir los cambios en caso de falla en el query
except Exception as e:
    print(f"Error: {e}")
    cursor.connection.rollback()

query2_avg_life_continente

,continent,avg_life_expectancy
0,Europe,75.147728
1,North America,72.991892
2,South America,70.946153
3,Oceania,69.715001
4,Asia,67.441177
5,Africa,52.571930


#### Consulta 3 - Porcentaje del area territorio en el continente de America que se habla cada Idioma oficial

In [52]:
try:
    # consulta
    query3 = """
    SELECT language, area, tabla_idiomas.area/tabla_total.total_area*100 AS "%_area"
    FROM (
        SELECT language, SUM(surface_area) AS area
        FROM country
        LEFT JOIN country_language ON country.code = country_language.country_code
        WHERE is_official=TRUE AND continent LIKE '%America%'
        GROUP BY language
        ORDER BY area DESC) AS tabla_idiomas,
    (SELECT SUM(surface_area) AS total_area
    FROM country
    WHERE continent LIKE '%America%') AS tabla_total
    LIMIT ;
    """
    # ejecutar consulta
    cursor.execute(query3)
    # extraer todas la filas de consulta
    res3 = cursor.fetchall()
    # guardar en dataframe
    query3_porc_area_idioma = pd.DataFrame(res3, columns=[desc[0] for desc in cursor.description])

# Excepcion para revertir los cambios en caso de falla en el query
except Exception as e:
    print(f"Error: {e}")
    cursor.connection.rollback()

query3_porc_area_idioma

Error: syntax error at or near ";"
LINE 13:     LIMIT ;
                   ^



,language,area,%_area
0,English,19372586.0,46.038175
1,Spanish,11462199.0,27.239457
2,French,10001409.0,23.767948
3,Portuguese,8547403.0,20.312561
4,Ketua,2383797.0,5.664998


In [53]:
engine = psycopg2.connect(
dbname="postgres",
user="postgres",
password="uNicentro17",
host="sofia.cmubcp0881c7.us-east-1.rds.amazonaws.com",
port='5432'
)

In [54]:
engine = psycopg2.connect(
dbname="world",
user="postgres",
password="uNicentro17",
host="sofia.cmubcp0881c7.us-east-1.rds.amazonaws.com",
port='5432'
)
cursor = engine.cursor()

# Consulta 1: Número total de ciudades
cursor.execute("SELECT COUNT(*) FROM city;")
numero_ciudades = cursor.fetchone()[0]
print("Número total de ciudades:", numero_ciudades)

# Consulta 2: Ciudad con la mayor población
cursor.execute("SELECT name, population FROM city ORDER BY population DESC LIMIT 1;")
ciudad_mas_poblada = cursor.fetchone()
print("Ciudad con mayor población:", ciudad_mas_poblada)

# Consulta 3: Número de ciudades por país
cursor.execute("SELECT country_code, COUNT(*) FROM city GROUP BY country_code;")
ciudades_por_pais = cursor.fetchall()
print("Número de ciudades por país:")
for pais in ciudades_por_pais:
    print(f"País: {pais[0]}, Número de ciudades: {pais[1]}")

cursor.close()
engine.close()



Número total de ciudades: 4079
Ciudad con mayor población: ('Mumbai (Bombay)', 10500000)
Número de ciudades por país:
País: MDG, Número de ciudades: 5
País: PLW, Número de ciudades: 1
País: BMU, Número de ciudades: 2
País: PSE, Número de ciudades: 6
País: COG, Número de ciudades: 2
País: CAF, Número de ciudades: 1
País: GMB, Número de ciudades: 2
País: SOM, Número de ciudades: 3
País: HKG, Número de ciudades: 2
País: URY, Número de ciudades: 1
País: BEL, Número de ciudades: 9
País: MRT, Número de ciudades: 2
País: BTN, Número de ciudades: 1
País: CZE, Número de ciudades: 10
País: ERI, Número de ciudades: 1
País: SVN, Número de ciudades: 2
País: LTU, Número de ciudades: 5
País: CIV, Número de ciudades: 5
País: PHL, Número de ciudades: 136
País: MLT, Número de ciudades: 2
País: GUM, Número de ciudades: 2
País: TCA, Número de ciudades: 1
País: ESP, Número de ciudades: 59
País: TUN, Número de ciudades: 8
País: GHA, Número de ciudades: 5
País: BLR, Número de ciudades: 16
País: SGP, Número d

In [55]:
engine = psycopg2.connect(
dbname="world",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)

cursor = engine.cursor()

# Consulta 1: Número de distritos
cursor.execute("SELECT COUNT(DISTINCT district) FROM city;")
numero_distritos = cursor.fetchone()[0]
print("Número total de distritos:", numero_distritos)

# Consulta 2: Ciudad con la menor población
cursor.execute("SELECT name, population FROM city ORDER BY population ASC LIMIT 1;")
ciudad_menor_poblacion = cursor.fetchone()
print("Ciudad con menor población:", ciudad_menor_poblacion)

# Consulta 3: Población total por distrito
cursor.execute("SELECT district, SUM(population) FROM city GROUP BY district ORDER BY SUM(population) DESC;")
poblacion_por_distrito = cursor.fetchall()
print("Población total por distrito:")
for distrito in poblacion_por_distrito:
    print(f"Distrito: {distrito[0]}, Población total: {distrito[1]}")


Número total de distritos: 1367
Ciudad con menor población: ('Adamstown', 42)
Población total por distrito:
Distrito: São Paulo, Población total: 26316966
Distrito: Maharashtra, Población total: 23659433
Distrito: England, Población total: 19978543
Distrito: Punjab, Población total: 19708438
Distrito: California, Población total: 16716706
Distrito: Distrito Federal, Población total: 15635629
Distrito: Liaoning, Población total: 15079174
Distrito: Uttar Pradesh, Población total: 14291064
Distrito: West Bengali, Población total: 12121743
Distrito: Shandong, Población total: 12114416
Distrito: Rio de Janeiro, Población total: 11958471
Distrito: Heilongjiang, Población total: 11628057
Distrito: Tokyo-to, Población total: 11043967
Distrito: Sindh, Población total: 11020081
Distrito: Buenos Aires, Población total: 10530136
Distrito: Andhra Pradesh, Población total: 10290295
Distrito: México, Población total: 10181634
Distrito: Seoul, Población total: 9981619
Distrito: Tamil Nadu, Población t

---
***Evidencia Código de Python (sofia)**


In [56]:
import psycopg2

In [57]:
engine = psycopg2.connect(
dbname="postgres",
user="postgres",
password="uNicentro17",
host="sofia.cmubcp0881c7.us-east-1.rds.amazonaws.com",
port='5432'
)

In [58]:
engine = psycopg2.connect(
dbname="world",
user="postgres",
password="uNicentro17",
host="sofia.cmubcp0881c7.us-east-1.rds.amazonaws.com",
port='5432'
)
cursor = engine.cursor()

# Consulta 1: Número total de ciudades
cursor.execute("SELECT COUNT(*) FROM city;")
numero_ciudades = cursor.fetchone()[0]
print("Número total de ciudades:", numero_ciudades)

# Consulta 2: Ciudad con la mayor población
cursor.execute("SELECT name, population FROM city ORDER BY population DESC LIMIT 1;")
ciudad_mas_poblada = cursor.fetchone()
print("Ciudad con mayor población:", ciudad_mas_poblada)

# Consulta 3: Número de ciudades por país
cursor.execute("SELECT country_code, COUNT(*) FROM city GROUP BY country_code;")
ciudades_por_pais = cursor.fetchall()
print("Número de ciudades por país:")
for pais in ciudades_por_pais:
    print(f"País: {pais[0]}, Número de ciudades: {pais[1]}")

cursor.close()
engine.close()



Número total de ciudades: 4079
Ciudad con mayor población: ('Mumbai (Bombay)', 10500000)
Número de ciudades por país:
País: MDG, Número de ciudades: 5
País: PLW, Número de ciudades: 1
País: BMU, Número de ciudades: 2
País: PSE, Número de ciudades: 6
País: COG, Número de ciudades: 2
País: CAF, Número de ciudades: 1
País: GMB, Número de ciudades: 2
País: SOM, Número de ciudades: 3
País: HKG, Número de ciudades: 2
País: URY, Número de ciudades: 1
País: BEL, Número de ciudades: 9
País: MRT, Número de ciudades: 2
País: BTN, Número de ciudades: 1
País: CZE, Número de ciudades: 10
País: ERI, Número de ciudades: 1
País: SVN, Número de ciudades: 2
País: LTU, Número de ciudades: 5
País: CIV, Número de ciudades: 5
País: PHL, Número de ciudades: 136
País: MLT, Número de ciudades: 2
País: GUM, Número de ciudades: 2
País: TCA, Número de ciudades: 1
País: ESP, Número de ciudades: 59
País: TUN, Número de ciudades: 8
País: GHA, Número de ciudades: 5
País: BLR, Número de ciudades: 16
País: SGP, Número d

In [59]:
engine = psycopg2.connect(
dbname="world",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)

cursor = engine.cursor()

# Consulta 1: Número de distritos
cursor.execute("SELECT COUNT(DISTINCT district) FROM city;")
numero_distritos = cursor.fetchone()[0]
print("Número total de distritos:", numero_distritos)

# Consulta 2: Ciudad con la menor población
cursor.execute("SELECT name, population FROM city ORDER BY population ASC LIMIT 1;")
ciudad_menor_poblacion = cursor.fetchone()
print("Ciudad con menor población:", ciudad_menor_poblacion)

# Consulta 3: Población total por distrito
cursor.execute("SELECT district, SUM(population) FROM city GROUP BY district ORDER BY SUM(population) DESC;")
poblacion_por_distrito = cursor.fetchall()
print("Población total por distrito:")
for distrito in poblacion_por_distrito:
    print(f"Distrito: {distrito[0]}, Población total: {distrito[1]}")


Número total de distritos: 1367
Ciudad con menor población: ('Adamstown', 42)
Población total por distrito:
Distrito: São Paulo, Población total: 26316966
Distrito: Maharashtra, Población total: 23659433
Distrito: England, Población total: 19978543
Distrito: Punjab, Población total: 19708438
Distrito: California, Población total: 16716706
Distrito: Distrito Federal, Población total: 15635629
Distrito: Liaoning, Población total: 15079174
Distrito: Uttar Pradesh, Población total: 14291064
Distrito: West Bengali, Población total: 12121743
Distrito: Shandong, Población total: 12114416
Distrito: Rio de Janeiro, Población total: 11958471
Distrito: Heilongjiang, Población total: 11628057
Distrito: Tokyo-to, Población total: 11043967
Distrito: Sindh, Población total: 11020081
Distrito: Buenos Aires, Población total: 10530136
Distrito: Andhra Pradesh, Población total: 10290295
Distrito: México, Población total: 10181634
Distrito: Seoul, Población total: 9981619
Distrito: Tamil Nadu, Población t